# Unit 5 Lecture 2 - Wrangling Data

ESI4628: Decision Support Systems for Industrial Engineers<br>
University of Central Florida
Dr. Ivan Garibay, Ramya Akula, Mostafa Saeidi, Madeline Schiappa, and Brett Belcher. 
https://github.com/igaribay/DSSwithPython/blob/master/DSS-Week05/Notebook/DSS-Unit05-Lecture02.2018.ipynb

## Notebook Learning Objectives
After studying this notebook students should be able to:
- Reshaping data using stack and unstack index data hierarchically
- aggregate, group, filter and transform data
- merge datasets using inner, outer, left and right join operations
- create Pivot tables
- remove NaN from data and remove duplicates

# Overview

When the large amounts of data are spread across various files, accessing that data is lot easier when organized properly. In this chapter we will learn how to aggregate the data by wrangling as needed. 

https://datahub.io/core/s-and-p-500-companies#python


# Reshaping with Hierarchical Indexing

Dealing with higher dimensional data is always a challenge, so we need to index them hierarchically, this helps in working with higher dimensional data in lower dimensional form, also known as hierarchical indexing. Lets begin with an simple example. 

In [1]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9), index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd' ], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.443497
   2    0.782177
   3   -0.414863
b  1    0.383333
   3   -0.406066
c  1    1.090454
   2    1.768006
d  2    0.168077
   3    1.201787
dtype: float64

In [81]:
data.unstack() # Series with two indexes becomes a DataFrame with one index

,1,2,3
a,-2.016269,1.235120,0.667338
b,0.108085,NaN,1.531259
c,-1.342022,0.900913,NaN
d,NaN,0.521124,0.223445


In [82]:
data.unstack().stack() # operation is reversible

a  1   -2.016269
   2    1.235120
   3    0.667338
b  1    0.108085
   3    1.531259
c  1   -1.342022
   2    0.900913
d  2    0.521124
   3    0.223445
dtype: float64

Hierarchical indexing plays an vital role in reshaping the data and group based operations like forming a pivot table. Below are some of the methods used to reorganize the data in python.

data.unstack() : rearrange data into a dataframe using its unstack method.
data.unstack().stack() : inverse operation of unstack. 

With a dataframe, either axis can have a hierarchical index


In [83]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [84]:
frame.unstack() #by default unstack inner row,  index=1 in this case

Ohio            Colorado    
  Green    Red        Green    
      1  2   1   2        1   2
a     0  3   1   4        2   5
b     6  9   7  10        8  11

In [87]:
frame.unstack(0) #but we can indicate to unstack other row index=0

Ohio            Colorado    
  Green    Red        Green    
      a  b   a   b        a   b
1     0  6   1   7        2   8
2     3  9   4  10        5  11

Below are the methods used to rearrange the order of the levels on an axis or sort the data by the values in one specific level. 

* frame.swaplevel() : The swaplevel takes two level numbers or names and returns a new object with the levels interchanged. 
* frame.sort_index() : This method sorts the data using only the values in a single level. <br>
** Note: ** When swapping levels, it is not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level. For instance, <code>frame.swaplevel(0,1).sort_index(level=0)</code>. 


In [93]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [94]:
frame.swaplevel(0,1)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
2 a     3   4        5
1 b     6   7        8
2 b     9  10       11

In [95]:
frame.swaplevel(0,1).sort_index(level=0)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
  b     6   7        8
2 a     3   4        5
  b     9  10       11

# Aggregate, Group, Filter and Transform Data

Aggregation Methods:
* df.count() : This Method counts total number of items
* df.first(), df.last() : Methods to get first and last item respectively.
* df.mean(), df.median() : Methods to get mean and median respectively.
* df. min(), df.max() : Methods to get minimum and maximum values respectively.
* df.std(), df.var(): Methods to get standard deviation and variance respectively.
* df.mad() : Method to get mean absolute deviation 
* df.prod() : Method to get product of the all items
* df.sum() : Method to get sum of all the items.

Groupby: Split, Apply ,Combine
* Split : Breaks and groups data frame depending on the value of the specified key.
* Apply : Computes some functions, usually an aggregate, transformation or filtering, within individual groups.
* Combine : Merges the results of these operations into an output array.

In [96]:
import numpy as np
rng = np.random.RandomState(0)
df = pd.DataFrame({'key' : [ 'A', 'B', 'C', 'A', 'B', 'C'], 'data1': range(6), 'data2': rng.randint(0, 10, 6)})
df


,data1,data2,key
0,0,5,A
1,1,0,B
2,2,3,C
3,3,3,A
4,4,7,B
5,5,9,C


In [43]:
#Aggregation
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [44]:
#Grouping by key using the mean to aggregate
df.groupby('key').mean()

,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


In [45]:
#Filtering by key using mean. Only include data if data2 mean for a group (A,B,C) is more than or equal to 4.
def filter_func(x):
    return x['data2'].mean() >= 4

df.groupby('key').filter(filter_func)

,data1,data2,key
0,0,5,A
2,2,3,C
3,3,3,A
5,5,9,C


While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine.

In [46]:
# Transformation

df.groupby('key').transform(lambda x: x - x.mean() )



,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


**Apply()** : <br>

This method lets us apply an arbitrary function to the group results. This method should take a data frame and return either a Pandas object(e.g., DataFrame series) or a scalar; 



In [47]:
df

,data1,data2,key
0,0,5,A
1,1,0,B
2,2,3,C
3,3,3,A
4,4,7,B
5,5,9,C


In [48]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

,data1,data2,key
0,0.000000,5,A
1,0.142857,0,B
2,0.166667,3,C
3,0.375000,3,A
4,0.571429,7,B
5,0.416667,9,C


# Merge and Join Data

To combine datasets by linking rows using one or more keys, we use **Merge** or **Join** operations. <br>

In [100]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a',  'b'], 'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [101]:
df2 = pd.DataFrame ({'key': [ 'a', 'b', 'd'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [51]:

pd.merge(df1, df2) 

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


This **Merge** merges by using the overlapping column names as the keys. This is because we did not specify which column to **join** explicitly. So to overcome this problem we use 



In [52]:
pd.merge(df1, df2, on = 'key')


,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


By default the behaviour of **join** is inner here, however that can also be modified as follows:

Other options : <br>
* Inner : Use only the key combinations observed in both tables.
* Outer : Use all the key combinations observed in both tables together.
* Left : Use all key combinations found in the left table.
* Right : Use all key combinations found in the right table.


In [53]:
pd.merge(df1, df2, how = 'outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [99]:
pd.merge(df1, df2, how = 'right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


Concatenation is another operation used for data combination. This operation can be performed using numpy function as below:

In [54]:
arr = np.arange(12).reshape((3, 4))
print(arr)
np.concatenate([arr, arr], axis =1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

The concat function in pandas provides a consistent way, specially when there is more than one axis which insufficient data.



In [55]:
s1 = pd.Series([0,1], index = ['a', 'b'])
s2 = pd.Series([ 2, 3, 4], index = ['c', 'd', 'e'])
s3 = pd.Series([5, 6], index = ['f', 'g'] )
print('S1 : ')
print(s1)
print('S2 : ')
print(s2)
print('S3 : ')
print(s3)


S1 : 
a    0
b    1
dtype: int64
S2 : 
c    2
d    3
e    4
dtype: int64
S3 : 
f    5
g    6
dtype: int64


In [56]:
pd.concat([s1, s2, s3], axis = 1, sort=True)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Now let us define the behaviour of **join**

In [57]:
s4 = pd.concat([s1, s3])
pd.concat([s1, s4], axis = 1, join = 'inner')


,0,1
a,0,0
b,1,1


# Pivot Tables

A pivot table is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The pivot table takes simple column-wise data as input and groups the entries into a two dimensional table that provides a multidimensional summarization of the data. <br>

In [58]:
import numpy as np
import pandas as pd
import seaborn as sns # statistical data visualization
titanic = sns.load_dataset('titanic') # load_dataset looks for online csv files 
                                    #on https://github.com/mwaskom/seaborn-data
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [59]:
titanic.pivot_table(values=['survived','fare'], index = 'sex', columns = 'class')

fare                        survived                    
class        First     Second      Third     First    Second     Third
sex                                                                   
female  106.125798  21.970121  16.118810  0.968085  0.921053  0.500000
male     67.226127  19.741782  12.661633  0.368852  0.157407  0.135447

The aggfunc keyword controls what type of aggregation is applied, which is mean by default. As in the GroupBy, the aggregation specification can be a string representing one of several common choices(‘sum’, ‘mean’, ‘count’, ‘min’, ‘max’, etc.) or a function that implements an aggregation (np.sum(), min(), sum(), etc.).

Additionally, it can be specified as dictionary mapping a column to any of the above desired options: 

In [60]:
titanic.pivot_table(index = 'sex', columns = 'class', aggfunc = {'survived' : sum, 'fare' : 'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

# Data cleaning and preparation

Data cleaning is the process of removing bad data in a dataset. This bad data includes incorrect and improperly formatted data as well as duplicated and missing data. 

In [61]:
# Example of a student survey dataset which includes incorrect and improperly formatted data.

csv_path = 'https://s3.amazonaws.com/dss-fall2018/Student_Survey.csv'

df = pd.read_csv (csv_path)
df

,Year,Location,Education,Sample_Size,Satisfactory
0,2017.0,Putnam,Middle School,659.0,Y
1,2018.0,Lexington,Middle School,649.0,N
2,2018.0,Lexington,Middle School,435.0,N
3,2017.0,Berkeley,NaN,NaN,NaN
4,NaN,Berkeley,High School,228.0,Y
5,2018.0,Berkeley,Middle School,20.0,NaN
6,2018.0,Washington,High School,437.0,N
7,NaN,Tremont,High School,NaN,Y
8,2016.0,Tremont,High School,220.0,Y


Let's take a look at the dataset. There are seven NA values in all columns. By using ```isnull``` function, pandas recognizes all missing value and return ```True```  

In [62]:
# Recognizing missing values

print (df.isnull())


    Year  Location  Education  Sample_Size  Satisfactory
0  False     False      False        False         False
1  False     False      False        False         False
2  False     False      False        False         False
3  False     False       True         True          True
4   True     False      False        False         False
5  False     False      False        False          True
6  False     False      False        False         False
7   True     False      False         True         False
8  False     False      False        False         False


### ```dropna``` method

```dropna``` is a method to filter missing data. Sometimes you need to work on only correct data and want to omit others.  

In [63]:
# Dropping all missing data by omitting rows and columns which include missing data

df.dropna()

,Year,Location,Education,Sample_Size,Satisfactory
0,2017.0,Putnam,Middle School,659.0,Y
1,2018.0,Lexington,Middle School,649.0,N
2,2018.0,Lexington,Middle School,435.0,N
6,2018.0,Washington,High School,437.0,N
8,2016.0,Tremont,High School,220.0,Y


In [65]:
# Dropping rows and columns which are all NA (In this example, there is no row includes all NA)

df.dropna(how = 'all')    #For rows
df.dropna(axis = 1, how ='all')     #For columns

,Year,Location,Education,Sample_Size,Satisfactory
0,2017.0,Putnam,Middle School,659.0,Y
1,2018.0,Lexington,Middle School,649.0,N
2,2018.0,Lexington,Middle School,435.0,N
3,2017.0,Berkeley,NaN,NaN,NaN
4,NaN,Berkeley,High School,228.0,Y
5,2018.0,Berkeley,Middle School,20.0,NaN
6,2018.0,Washington,High School,437.0,N
7,NaN,Tremont,High School,NaN,Y
8,2016.0,Tremont,High School,220.0,Y


### ```fillna``` method 

```fillna``` is a method to fill missing data by any number or value.  

In [66]:
# filling missing data by '100' in 'Sample_Size' column. 

print (df['Sample_Size'])
df['Sample_Size'].fillna(100)

0    659.0
1    649.0
2    435.0
3      NaN
4    228.0
5     20.0
6    437.0
7      NaN
8    220.0
Name: Sample_Size, dtype: float64


0    659.0
1    649.0
2    435.0
3    100.0
4    228.0
5     20.0
6    437.0
7    100.0
8    220.0
Name: Sample_Size, dtype: float64

In [67]:
# filling missing data in column 'Year' by 2015.

df.fillna({'Year': 2015})

,Year,Location,Education,Sample_Size,Satisfactory
0,2017.0,Putnam,Middle School,659.0,Y
1,2018.0,Lexington,Middle School,649.0,N
2,2018.0,Lexington,Middle School,435.0,N
3,2017.0,Berkeley,NaN,NaN,NaN
4,2015.0,Berkeley,High School,228.0,Y
5,2018.0,Berkeley,Middle School,20.0,NaN
6,2018.0,Washington,High School,437.0,N
7,2015.0,Tremont,High School,NaN,Y
8,2016.0,Tremont,High School,220.0,Y


### Removing Duplicates

Sometimes in a DataFrame, you have duplicate rows and you need to remove duplicate.

For having a sample of DataFrame with duplicate rows, let's duplicate one of the rows in the previous example (like row number 6):

In [72]:
# Creating a DataFrame with two same rows
raw_data = [['Bruce','Banner',38,4,25],['Tony','Stark',42,24,94],['Hal','Jordan',25,31,57],['Bruce','Wayne',32,2,62],
            ['Clark','Kent',28,3,70],['Tony','Stark',42,24,94]]
df = pd.DataFrame (raw_data, columns = ['first_name', 'last_name','age','preTestScore','postTestScore'])
df

,first_name,last_name,age,preTestScore,postTestScore
0,Bruce,Banner,38,4,25
1,Tony,Stark,42,24,94
2,Hal,Jordan,25,31,57
3,Bruce,Wayne,32,2,62
4,Clark,Kent,28,3,70
5,Tony,Stark,42,24,94


```duplicated()``` method returns a boolean value whether each row is a duplicate. In this DataFrame, the value for number 5 is 'True' and says row number 5 is duplicated.

In [73]:
df.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
dtype: bool

```drop_duplicates()``` method returns DataFrame with duplicate rows removed

In [74]:
df.drop_duplicates()

,first_name,last_name,age,preTestScore,postTestScore
0,Bruce,Banner,38,4,25
1,Tony,Stark,42,24,94
2,Hal,Jordan,25,31,57
3,Bruce,Wayne,32,2,62
4,Clark,Kent,28,3,70


# References
1. Seaborn, statistical data visualization, https://seaborn.pydata.org
2. Data repo for Seaborn examples, https://github.com/mwaskom/seaborn-data
3. Pivot Tables, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html
4. Reshaping, Pivot, Stack, unstack, https://pandas.pydata.org/pandas-docs/stable/reshaping.html